In [1]:
import sys

sys.path.append("../")

DATA_PATH = '/mnt/d/Working/PersonalProjects/Trading/trading-agent/crypto-pair-trading/data/crypto/'

In [2]:
import pandas as pd
import numpy as np
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from stats_arb.tests import adf_test, kpss_test, cal_half_life, pp_test
from datetime import datetime, timedelta
import itertools as it
import backtrader as bt
import requests
import empyrical
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 6) # (w, h)
plt.ioff()

In [3]:
def zscore(x: pd.Series, window: int):
    r = x.rolling(window=window)
    m = r.mean().shift(1)
    s = r.std(ddof=0).shift(1)
    z = (x-m)/s
    return z

In [4]:
API_BASE = 'https://fapi.binance.com/fapi/v1/'

LABELS = [
    'open_time',
    'open',
    'high',
    'low',
    'close',
    'volume',
    'close_time',
    'quote_asset_volume',
    'number_of_trades',
    'taker_buy_base_asset_volume',
    'taker_buy_quote_asset_volume',
    'ignore'
]

DROP_COLUMNS=[
    'close_time',
    'quote_asset_volume',
    'number_of_trades',
    'taker_buy_base_asset_volume',
    'taker_buy_quote_asset_volume',
    'ignore'
]


def get_batch(symbol, interval='1m', start_time=0, limit=1000):
    """Use a GET request to retrieve a batch of candlesticks. Process the JSON into a pandas
    dataframe and return it. If not successful, return an empty dataframe.
    """

    params = {
        'symbol': symbol,
        'interval': interval,
        'startTime': start_time,
        'limit': limit
    }
    try:
        # timeout should also be given as a parameter to the function
        response = requests.get(f'{API_BASE}klines', params, timeout=30)
    except requests.exceptions.ConnectionError:
        print('Connection error, Cooling down for 5 mins...')
        time.sleep(15)
        return get_batch(symbol, interval, start_time, limit)

    except requests.exceptions.Timeout:
        print('Timeout, Cooling down for 5 min...')
        time.sleep(15)
        return get_batch(symbol, interval, start_time, limit)

    if response.status_code == 200:
        return pd.DataFrame(response.json(), columns=LABELS)
    
    print(f'Got erroneous response back {symbol}: {response}. {response.text}')
    return pd.DataFrame([])


def get_candles(base, quote, start_date: datetime, interval='1m'):
    batches = []

    last_timestamp = int(start_date.timestamp()) * 1000
    # gather all candlesticks available, starting from the last timestamp loaded from disk or 0
    # stop if the timestamp that comes back from the api is the same as the last one
    previous_timestamp = None

    while previous_timestamp != last_timestamp:
        # stop if we reached data from today
        if datetime.fromtimestamp(last_timestamp / 1000) >= datetime.utcnow():
            break

        previous_timestamp = last_timestamp

        new_batch = get_batch(
            symbol=base + quote,
            interval=interval,
            start_time=last_timestamp
        )

        # requesting candles from the future returns empty
        # also stop in case response code was not 200
        if new_batch.empty:
            break

        last_timestamp = new_batch['open_time'].max()

        # sometimes no new trades took place yet on date.today();
        # in this case the batch is nothing new
        if previous_timestamp == last_timestamp:
            break

        batches.append(new_batch)
        last_datetime = datetime.fromtimestamp(last_timestamp / 1000)

        covering_spaces = 20 * ' '
        print(datetime.now(), base, quote, interval, str(last_datetime) + covering_spaces, end='\r', flush=True)

    if len(batches) > 0:
        # write clean version of csv to parquet
        df = pd.concat(batches, ignore_index=True)
        df.drop(columns=DROP_COLUMNS, inplace=True)
        df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
        df.set_index('open_time', inplace=True)
        df = df[~df.index.duplicated(keep='first')]
        df = df.astype('float32')
        return df



In [5]:
def print_analysis(analyzer):
    """
    Function to print the Technical Analysis results in a nice format.
    """
    # Get the results we are interested in
    total_open = analyzer.total.open
    total_closed = analyzer.total.closed
    total_won = analyzer.won.total
    total_lost = analyzer.lost.total
    win_streak = analyzer.streak.won.longest
    lose_streak = analyzer.streak.lost.longest
    pnl_net = round(analyzer.pnl.net.total, 2)
    strike_rate = (total_won / total_closed) * 100
    # Designate the rows
    h1 = ["Total Open", "Total Closed", "Total Won", "Total Lost"]
    h2 = ["Strike Rate", "Win Streak", "Losing Streak", "PnL Net"]
    r1 = [total_open, total_closed, total_won, total_lost]
    r2 = [strike_rate, win_streak, lose_streak, pnl_net]
    # Check which set of headers is the longest.
    if len(h1) > len(h2):
        header_length = len(h1)
    else:
        header_length = len(h2)
    # Print the rows
    print_list = [h1, r1, h2, r2]
    row_format = "{:<15}" * (header_length + 1)
    print("Trade Analysis Results:")
    for row in print_list:
        print(row_format.format("", *row))

In [6]:
data = {}

# config

In [7]:

timeframe = '15m'
stationary_df = pd.read_csv(f'stationary_df_{timeframe}.csv')
stationary_df

,i,pairs,half_life,hedge_ratio,sid_1,sid_2
0,12,['LTC' 'XMR'],40,[ 26.26102528 -70.21746029],LTC,XMR
1,34,['MATIC' 'XMR'],49,[22.88928805 35.38803656],MATIC,XMR
2,39,['XMR' 'UNI'],53,[37.72579497 18.23129893],XMR,UNI
3,41,['MANA' 'UNI'],61,[ 49.52048185 -34.10343683],MANA,UNI
4,40,['NEAR' 'MANA'],61,[ 28.55084246 -50.96965568],NEAR,MANA
5,27,['BNB' 'XMR'],62,[20.62332548 36.70043684],BNB,XMR
6,9,['XRP' 'XMR'],63,[ 4.83818405 -48.98621273],XRP,XMR
7,38,['XMR' 'MANA'],65,[27.97782191 18.06431601],XMR,MANA
8,4,['BTC' 'XMR'],69,[ 10.67405194 -49.21769878],BTC,XMR
9,11,['LTC' 'MATIC'],69,[14.58328594 26.82504414],LTC,MATIC


In [8]:
# # Get the dates from the args
# today = datetime.now()
# if timeframe == '1h':
#     fromdate = today - timedelta(days=4 * 30)
#     start_trading_at = today - timedelta(days=30)
# else:
#     fromdate = today - timedelta(days=20)
#     start_trading_at = today - timedelta(days=20)

# nb_symbols = len(hedge_ratio)
# pair = [selected_row[f'sid_{i + 1}'] for i in range(nb_symbols)]
# print(pair)
# position_size = 5

# for symbol in pair:
#     df_ = get_candles(base=symbol, quote='USDT', start_date=fromdate, interval=timeframe)
#     data[symbol] = df_

# start_trading_at

In [9]:

def convert_hedge_ratio(hedge_ratio):
    hedge_ratio = hedge_ratio.replace('[', '')
    hedge_ratio = hedge_ratio.replace(']', '')
    hedge_ratio = hedge_ratio.strip()
    hedge_ratio = [float(x) for x in hedge_ratio.split(' ') if x != '']
    return hedge_ratio


# Get the dates from the args
today = datetime.now()
if timeframe == '1h':
    fromdate = today - timedelta(days=4 * 30)
    start_trading_at = today - timedelta(days=60)
elif timeframe in ['15m']:
    fromdate = today - timedelta(days=30)
    start_trading_at = today - timedelta(days=30)
elif timeframe in ['1d']:
    fromdate = today - timedelta(days=6*30)
    start_trading_at = today - timedelta(days=252)
else:
    fromdate = today - timedelta(days=45)
    start_trading_at = today - timedelta(days=30)

# fromdate = datetime(2023, 1, 2) - timedelta(days=30)
# start_trading_at = datetime(2023, 1, 2) - timedelta(days=30)


In [16]:
# class Strategy(bt.Strategy):
#     params = dict(
#         symbols=[],
#         hedge_ratio=[],
#         half_life=None,
#         nb_symbols=2,
#         sl_upper_limit=10,
#         middle=0,
#         upper_limit=3,
#         upper_limit2=5,
#         lower_limit=-3,
#         lower_limit2=-5,
#         sl_lower_limit=-10,
#         position_size=10,
#         start_trading_at=None,
#         dca_enabled=True,
#         fixed_weight=True,
#         debug=False,
#     )

#     def __init__(self) -> None:
#         super().__init__()
#         self.portfolio = {d._name: d for d in self.datas if d._name in self.p.symbols}
#         self.lookback = self.p.half_life
#         self.status = 0
#         self.in_position = False
#         self.dca_count = 0
#         self.hedge_ratio = self.p.hedge_ratio
#         self.half_life = self.p.half_life
#         self.fixed_weight = self.p.fixed_weight
#         self.log(
#             f"Start trade with {self.p.symbols}, {self.p.hedge_ratio}, {self.p.half_life}, {self.p.nb_symbols}"
#         )

#     def log(self, txt, dt=None):
#         dt = dt or self.data.datetime[0]
#         dt = bt.num2date(dt)
#         if self.p.debug:
#             print("%s, %s" % (dt.isoformat(), txt))

#     def notify_order(self, order):
#         if order.status in [bt.Order.Submitted, bt.Order.Accepted]:
#             return  # Await further notifications

#         if order.status == order.Completed:
#             if order.isbuy():
#                 buytxt = (
#                     f"BUY COMPLETE {order.data._name}, "
#                     f"price: {order.executed.price:.4f}, "
#                     f"size: {order.executed.size}, "
#                     f"cost {order.executed.price * order.executed.size:.4f}"
#                 )
#                 self.log(buytxt, order.executed.dt)
#             else:
#                 selltxt = (
#                     f"SELL COMPLETE {order.data._name}, "
#                     f"price: {order.executed.price:.4f}, "
#                     f"size: {order.executed.size}"
#                     f"cost {order.executed.price * order.executed.size:.4f}"
#                 )
#                 self.log(selltxt, order.executed.dt)

#         elif order.status in [order.Expired, order.Canceled, order.Margin]:
#             self.log("Order cancelled %s ," % order.Status[order.status])
#             pass  # Simply log

#     def notify_trade(self, trade):
#         if not trade.isclosed:
#             return

#         self.log("OPERATION PROFIT, GROSS %.4f, NET %.4f" % (trade.pnl, trade.pnlcomm))
#         self.log(f"ACCOUNT VALUE: {self.broker.get_value()}")

#     def get_price_series(self, symbol):
#         d = self.portfolio[symbol]
#         lookback = self.lookback * 3
#         close = d.close.get(size=lookback)
#         t = [bt.num2date(dt) for dt in d.datetime.get(size=lookback)]
#         s = pd.Series(np.log(close), index=t)

#         return s

#     def calculate_spread(self):
#         spread = None
#         for i in range(self.p.nb_symbols):
#             close_s = self.get_price_series(self.p.symbols[i])
#             if spread is None:
#                 spread = close_s * self.hedge_ratio[i]
#             else:
#                 spread += close_s * self.hedge_ratio[i]

#         spread.dropna(inplace=True)
#         return spread

#     def calculate_zscore(self, spread):
#         zscore_df = zscore(spread, self.lookback)
#         return zscore_df

#     def long_spread(self, size=None):
#         for i, symbol in enumerate(self.p.symbols):
#             if not self.fixed_weight:
#                 size = np.sign(self.hedge_ratio[i]) * (
#                     self.p.position_size / self.portfolio[symbol].close[0]
#                 )
#                 size = (self.hedge_ratio[i] * self.p.position_size) / self.portfolio[
#                     symbol
#                 ].close[0]
#             else:
#                 size = np.sign(self.hedge_ratio[i]) * (
#                     self.p.position_size / self.portfolio[symbol].close[0]
#                 )

#             if size > 0:
#                 self.log(
#                     f"Buy {symbol}, size = {size}, cost = {self.portfolio[symbol].close[0] * abs(size)}"
#                 )
#                 self.buy(data=self.portfolio[symbol], size=abs(size))
#             else:
#                 self.log(
#                     f"Sell {symbol}, size = {size}, cost = {self.portfolio[symbol].close[0] * abs(size)}"
#                 )
#                 self.sell(data=self.portfolio[symbol], size=abs(size))

#     def short_spread(self):
#         for i, symbol in enumerate(self.p.symbols):
#             if not self.fixed_weight:
#                 size = -np.sign(self.hedge_ratio[i]) * (
#                     self.p.position_size / self.portfolio[symbol].close[0]
#                 )
#                 size = (
#                     -(self.hedge_ratio[i] * self.p.position_size)
#                     / self.portfolio[symbol].close[0]
#                 )
#             else:
#                 size = -np.sign(self.hedge_ratio[i]) * (
#                     self.p.position_size / self.portfolio[symbol].close[0]
#                 )
#             if size > 0:
#                 self.log(
#                     f"Buy {symbol}, size = {size}, cost = {self.portfolio[symbol].close[0] * abs(size)}"
#                 )
#                 self.buy(data=self.portfolio[symbol], size=abs(size))
#             else:
#                 self.log(
#                     f"Sell {symbol}, size = {size}, cost = {self.portfolio[symbol].close[0] * abs(size)}"
#                 )
#                 self.sell(data=self.portfolio[symbol], size=abs(size))

#     def close_all(self):
#         for i, symbol in enumerate(self.p.symbols):
#             self.close(self.portfolio[symbol])
#         self.status = 0
#         self.in_position = False
#         self.dca_count = 0

#     def unrealized_pnl(self):
#         unrealizel_pnl = 0
#         for symbol in self.p.symbols:
#             pos = self.getposition(self.portfolio[symbol])
#             pnl = self.portfolio[symbol].close[0] - pos.price
#             unrealizel_pnl += pnl
#         return unrealizel_pnl

#     def next(self):
#         dt = self.data.datetime[0]
#         dt = bt.num2date(dt)
#         if self.p.start_trading_at is not None and dt <= self.p.start_trading_at:
#             return

#         spread = self.calculate_spread()
#         if len(spread) < self.lookback * 2:
#             return

#         # if adf_test(spread, verbose=False) > 0.05 and not self.in_position:
#         #     self.log(f'Spread is not stationary')
#         # #     # if not self.calculate_hedge_ratio():
#         # #     #     return
#         #     return

#         # if self.in_position:
#         #     if self.unrealized_pnl() > 5:
#         #         self.close_all()
#         #         self.log(f'Take profit unrealized pnl > 10')
#         #         return
#         #     elif self.unrealized_pnl() <= -self.broker.getvalue() * 0.1:
#         #         self.close_all()
#         #         self.log(f'Take profit unrealized pnl < -100')
#         #         return

#         zscore = self.calculate_zscore(spread).iloc[-1]
#         if (
#             (zscore >= self.p.upper_limit)
#             and (self.status != 1)
#             and not self.in_position
#         ):
#             self.log(
#                 f"Short spread zscore {zscore} > {self.p.upper_limit}, {self.data0.close[0]} - {self.data1.close[0]}"
#             )
#             self.short_spread()
#             self.status = 1  # The current status is "short the spread"
#             self.in_position = True
#         if (
#             (zscore >= self.p.upper_limit2)
#             and (self.status == 1)
#             and self.in_position
#             and self.dca_count == 0
#             and self.p.dca_enabled
#         ):
#             self.log(f"DCA Short spread zscore {zscore} > {self.p.upper_limit2}")
#             self.short_spread()
#             self.dca_count += 1
#         elif (
#             zscore < self.p.lower_limit and (self.status != 2) and not self.in_position
#         ):
#             self.log(
#                 f"Long spread zscore {zscore} > {self.p.lower_limit}, {self.data0.close[0]} - {self.data1.close[0]}"
#             )
#             self.long_spread()

#             self.status = 2
#             self.in_position = True
#         elif (
#             zscore <= self.p.lower_limit2
#             and (self.status == 2)
#             and self.in_position
#             and self.dca_count == 0
#             and self.p.dca_enabled
#         ):
#             self.log(f"DCA Long spread zscore {zscore} > {self.p.upper_limit2}")
#             self.long_spread()
#             self.dca_count += 1

#         elif self.getposition(self.data0) and (
#             (zscore <= self.p.middle and self.status == 1)
#             or (zscore >= -self.p.middle and self.status == 2)
#         ):
#             self.log(f"Take profit {zscore} <> 0")
#             self.close_all()

#         elif self.getposition(self.data0) and (
#             (zscore >= self.p.sl_upper_limit and self.status == 1)
#             or (zscore <= self.p.sl_lower_limit and self.status == 2)
#         ):
#             self.log(f"Stop Loss {zscore} <> +- 2")
#             self.close_all()


# def run(
#     data,
#     pair,
#     hedge_ratio,
#     half_life,
#     start_trading_at,
#     nb_symbols,
#     position_size,
#     fixed_weight=False,
#     debug=False,
#     upper_limit=2.5,
#     upper_limit2=4,
#     middle=1,
#     lower_limit=-2.5,
#     lower_limit2=-4
# ):
#     # Create a cerebro
#     cerebro = bt.Cerebro()

#     for symbol in pair:
#         df_ = data[symbol]
#         # Create the 1st data
#         data0 = bt.feeds.PandasData(
#             dataname=df_,
#             plot=False,
#         )

#         # Add the 1st data to cerebro
#         cerebro.adddata(data0, name=symbol)

#     # Add the strategy
#     cerebro.addstrategy(
#         Strategy,
#         symbols=pair,
#         hedge_ratio=hedge_ratio,
#         half_life=half_life,
#         nb_symbols=nb_symbols,
#         position_size=position_size,
#         sl_upper_limit=14,
#         upper_limit=upper_limit,
#         upper_limit2=upper_limit2,
#         middle=middle,
#         lower_limit=lower_limit,
#         lower_limit2=lower_limit2,
#         sl_lower_limit=-14,
#         dca_enabled=True,
#         start_trading_at=start_trading_at,
#         fixed_weight=fixed_weight,
#         debug=debug,
#     )

#     cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
#     cerebro.addanalyzer(bt.analyzers.TimeReturn, _name="return")
#     cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.0)
#     cerebro.addanalyzer(bt.analyzers.Returns)
#     cerebro.addanalyzer(bt.analyzers.DrawDown)

#     # Add the commission - only stocks like a for each operation
#     cerebro.broker.setcash(1000)
#     cerebro.broker.setcommission(commission=0.04 / 100, leverage=10)

#     print("Starting Portfolio Value: %.2f" % cerebro.broker.getvalue())
#     # And run it
#     results = cerebro.run()
#     first_strat = results[0]
#     print_analysis(first_strat.analyzers.ta.get_analysis())
#     print("Final Portfolio Value: %.2f" % cerebro.broker.getvalue())
#     # Print out the final result
#     print(
#         f"Norm. Annual Return: {results[0].analyzers.returns.get_analysis()['rnorm100']:.2f}%"
#     )
#     print(
#         f"Max Drawdown: {results[0].analyzers.drawdown.get_analysis()['max']['drawdown']:.2f}%"
#     )
#     return (
#         round(results[0].analyzers.returns.get_analysis()["rnorm100"], 4),
#         results[0].analyzers.drawdown.get_analysis()["max"]["drawdown"],
#         results[0],
#     )


In [35]:
class Strategy(bt.Strategy):
    params = dict(
        symbols=[],
        hedge_ratio=[],
        half_life=None,
        nb_symbols=2,
        sl_upper_limit=10,
        middle=0,
        upper_limit=3,
        upper_limit2=5,
        lower_limit=-3,
        lower_limit2=-5,
        sl_lower_limit=-10,
        position_size=10,
        start_trading_at=None,
        dca_enabled=True,
        fixed_weight=True,
        debug=False,
        enable_tp_sl=False
    )

    def __init__(self) -> None:
        super().__init__()
        self.portfolio = {d._name: d for d in self.datas if d._name in self.p.symbols}
        self.lookback = self.p.half_life
        self.status = 0
        self.in_position = False
        self.dca_count = 0
        self.hedge_ratio = self.p.hedge_ratio
        self.half_life = self.p.half_life
        self.fixed_weight = self.p.fixed_weight
        self.last_dca_level = None
        self.log(
            f"Start trade with {self.p.symbols}, {self.p.hedge_ratio}, {self.p.half_life}, {self.p.nb_symbols}"
        )

    def log(self, txt, dt=None):
        dt = dt or self.data.datetime[0]
        dt = bt.num2date(dt)
        if self.p.debug:
            print("%s, %s" % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [bt.Order.Submitted, bt.Order.Accepted]:
            return  # Await further notifications

        if order.status == order.Completed:
            if order.isbuy():
                buytxt = (
                    f"BUY COMPLETE {order.data._name}, "
                    f"price: {order.executed.price:.4f}, "
                    f"size: {order.executed.size}, "
                    f"cost {order.executed.price * order.executed.size:.4f}"
                )
                self.log(buytxt, order.executed.dt)
            else:
                selltxt = (
                    f"SELL COMPLETE {order.data._name}, "
                    f"price: {order.executed.price:.4f}, "
                    f"size: {order.executed.size}"
                    f"cost {order.executed.price * order.executed.size:.4f}"
                )
                self.log(selltxt, order.executed.dt)

        elif order.status in [order.Expired, order.Canceled, order.Margin]:
            self.log("Order cancelled %s ," % order.Status[order.status])
            pass  # Simply log

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log("OPERATION PROFIT, GROSS %.4f, NET %.4f" % (trade.pnl, trade.pnlcomm))
        self.log(f"ACCOUNT VALUE: {self.broker.get_value()}")

    def get_price_series(self, symbol):
        d = self.portfolio[symbol]
        lookback = self.lookback * 3
        close = d.close.get(size=lookback)
        t = [bt.num2date(dt) for dt in d.datetime.get(size=lookback)]
        s = pd.Series(np.log(close), index=t)

        return s

    def calculate_spread(self):
        spread = None
        for i in range(self.p.nb_symbols):
            close_s = self.get_price_series(self.p.symbols[i])
            if spread is None:
                spread = close_s * self.hedge_ratio[i]
            else:
                spread += close_s * self.hedge_ratio[i]

        spread.dropna(inplace=True)
        return spread

    def calculate_zscore(self, spread):
        zscore_df = zscore(spread, self.lookback)
        return zscore_df

    def long_spread(self, position_size=None):
        if position_size is None:
            position_size = self.p.position_size

        for i, symbol in enumerate(self.p.symbols):
            if not self.fixed_weight:
                size = np.sign(self.hedge_ratio[i]) * (
                    position_size / self.portfolio[symbol].close[0]
                )
                size = (self.hedge_ratio[i] * position_size) / self.portfolio[
                    symbol
                ].close[0]
            else:
                size = np.sign(self.hedge_ratio[i]) * (
                    position_size / self.portfolio[symbol].close[0]
                )

            if size > 0:
                self.log(
                    f"Buy {symbol}, size = {size}, cost = {self.portfolio[symbol].close[0] * abs(size)}"
                )
                self.buy(data=self.portfolio[symbol], size=abs(size))
            else:
                self.log(
                    f"Sell {symbol}, size = {size}, cost = {self.portfolio[symbol].close[0] * abs(size)}"
                )
                self.sell(data=self.portfolio[symbol], size=abs(size))

    def short_spread(self, position_size=None):
        if position_size is None:
            position_size = self.p.position_size

        for i, symbol in enumerate(self.p.symbols):
            if not self.fixed_weight:
                size = -np.sign(self.hedge_ratio[i]) * (
                    position_size / self.portfolio[symbol].close[0]
                )
                size = (
                    -(self.hedge_ratio[i] * position_size)
                    / self.portfolio[symbol].close[0]
                )
            else:
                size = -np.sign(self.hedge_ratio[i]) * (
                    position_size / self.portfolio[symbol].close[0]
                )
            if size > 0:
                self.log(
                    f"Buy {symbol}, size = {size}, cost = {self.portfolio[symbol].close[0] * abs(size)}"
                )
                self.buy(data=self.portfolio[symbol], size=abs(size))
            else:
                self.log(
                    f"Sell {symbol}, size = {size}, cost = {self.portfolio[symbol].close[0] * abs(size)}"
                )
                self.sell(data=self.portfolio[symbol], size=abs(size))

    def close_all(self):
        for i, symbol in enumerate(self.p.symbols):
            self.close(self.portfolio[symbol])
        self.status = 0
        self.in_position = False
        self.dca_count = 0

    def unrealized_pnl(self):
        unrealizel_pnl = 0
        for symbol in self.p.symbols:
            pos = self.getposition(self.portfolio[symbol])
            pnl = (self.portfolio[symbol].close[0] - pos.price) * pos.size
            unrealizel_pnl += pnl
        return unrealizel_pnl

    def next(self):
        dt = self.data.datetime[0]
        dt = bt.num2date(dt)
        if self.p.start_trading_at is not None and dt <= self.p.start_trading_at:
            return

        spread = self.calculate_spread()
        if len(spread) < self.lookback * 2:
            return

        # if adf_test(spread, verbose=False) > 0.05 and not self.in_position:
        #     self.log(f'Spread is not stationary')
        # #     # if not self.calculate_hedge_ratio():
        # #     #     return
        #     return

        if self.in_position and self.p.enable_tp_sl:
            if self.unrealized_pnl() > 5:
                self.close_all()
                self.log(f'Take profit unrealized pnl > 10')
                return
            elif self.unrealized_pnl() <= -self.broker.getvalue() * 0.02:
                self.close_all()
                self.log(f'Stop loss unrealized pnl < -{self.broker.getvalue() * 0.02}')
                return

        zscore = self.calculate_zscore(spread).iloc[-1]
        std = spread.rolling(self.lookback).std().iloc[-1]
        if (
            (zscore >= self.p.upper_limit)
            and (self.status != 1)
            and not self.in_position
        ):
            self.log(
                f"Short spread zscore {zscore} > {self.p.upper_limit}, {self.data0.close[0]} - {self.data1.close[0]}"
            )
            self.short_spread()
            self.status = 1  # The current status is "short the spread"
            self.in_position = True
            self.last_dca_level = spread.iloc[-1]
        elif (
            self.last_dca_level is not None
            and spread.iloc[-1] >= self.last_dca_level + 2 * std
            and (self.status == 1)
            and self.in_position
            and self.dca_count == 0
            and self.p.dca_enabled
        ):
            self.log(f"DCA {self.dca_count} Short spread zscore {spread.iloc[-1]} > {self.last_dca_level + 2 * std}")
            self.short_spread(position_size=self.p.position_size * 1.5)
            self.dca_count += 1
            self.last_dca_level = spread.iloc[-1]
        elif (
            self.last_dca_level is not None
            and spread.iloc[-1] >= self.last_dca_level + 2 * std
            and (self.status == 1)
            and self.in_position
            and self.dca_count == 1
            and self.p.dca_enabled
        ):
            self.log(f"DCA {self.dca_count} Short spread zscore {spread.iloc[-1]} > {self.last_dca_level + 2 * std}")
            self.short_spread(position_size=self.p.position_size * 2)
            self.dca_count += 1
            self.last_dca_level = spread.iloc[-1]
        elif (
            zscore < self.p.lower_limit and (self.status != 2) and not self.in_position
        ):
            self.log(
                f"Long spread zscore {zscore} > {self.p.lower_limit}, {self.data0.close[0]} - {self.data1.close[0]}"
            )
            self.long_spread()

            self.status = 2
            self.in_position = True
            self.last_dca_level = spread.iloc[-1]
        elif (
            self.last_dca_level is not None
            and spread.iloc[-1] <= self.last_dca_level - 2 * std
            and (self.status == 2)
            and self.in_position
            and self.dca_count == 0
            and self.p.dca_enabled
        ):
            self.log(f"DCA {self.dca_count} Long spread zscore {spread.iloc[-1]} < {self.last_dca_level - 2 * std}")
            self.long_spread(position_size=self.p.position_size * 1.5)
            self.dca_count += 1
            self.last_dca_level = spread.iloc[-1]
        elif (
            self.last_dca_level is not None
            and spread.iloc[-1] <= self.last_dca_level - 2 * std
            and (self.status == 2)
            and self.in_position
            and self.dca_count == 1
            and self.p.dca_enabled
        ):
            self.log(f"DCA {self.dca_count} Long spread zscore {spread.iloc[-1]} < {self.last_dca_level - 2 * std}")
            self.long_spread(position_size=self.p.position_size * 2)
            self.dca_count += 1
            self.last_dca_level = spread.iloc[-1]

        elif self.getposition(self.data0) and (
            (zscore <= self.p.middle and self.status == 1)
            or (zscore >= -self.p.middle and self.status == 2)
        ):
            self.log(f"Take profit {zscore} <> 0")
            self.close_all()

        elif self.getposition(self.data0) and (
            (zscore >= self.p.sl_upper_limit and self.status == 1)
            or (zscore <= self.p.sl_lower_limit and self.status == 2)
        ):
            self.log(f"Stop Loss {zscore} <> +- 2")
            self.close_all()


def run(
    data,
    pair,
    hedge_ratio,
    half_life,
    start_trading_at,
    nb_symbols,
    position_size,
    fixed_weight=False,
    debug=False,
    upper_limit=2.5,
    upper_limit2=4,
    middle=1,
    lower_limit=-2.5,
    lower_limit2=-4,
    enable_tp_sl=False
):
    # Create a cerebro
    cerebro = bt.Cerebro()

    for symbol in pair:
        df_ = data[symbol]
        # Create the 1st data
        data0 = bt.feeds.PandasData(
            dataname=df_,
            plot=False,
        )

        # Add the 1st data to cerebro
        cerebro.adddata(data0, name=symbol)

    # Add the strategy
    cerebro.addstrategy(
        Strategy,
        symbols=pair,
        hedge_ratio=hedge_ratio,
        half_life=half_life,
        nb_symbols=nb_symbols,
        position_size=position_size,
        sl_upper_limit=14,
        upper_limit=upper_limit,
        upper_limit2=upper_limit2,
        middle=middle,
        lower_limit=lower_limit,
        lower_limit2=lower_limit2,
        sl_lower_limit=-14,
        dca_enabled=True,
        start_trading_at=start_trading_at,
        fixed_weight=fixed_weight,
        debug=debug,
        enable_tp_sl=enable_tp_sl
    )

    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
    cerebro.addanalyzer(bt.analyzers.TimeReturn, _name="return")
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.0)
    cerebro.addanalyzer(bt.analyzers.Returns)
    cerebro.addanalyzer(bt.analyzers.DrawDown)

    # Add the commission - only stocks like a for each operation
    cerebro.broker.setcash(1000)
    cerebro.broker.setcommission(commission=0.04 / 100, leverage=10)

    print("Starting Portfolio Value: %.2f" % cerebro.broker.getvalue())
    # And run it
    results = cerebro.run()
    first_strat = results[0]
    print_analysis(first_strat.analyzers.ta.get_analysis())
    print("Final Portfolio Value: %.2f" % cerebro.broker.getvalue())
    # Print out the final result
    print(
        f"Norm. Annual Return: {results[0].analyzers.returns.get_analysis()['rnorm100']:.2f}%"
    )
    print(
        f"Max Drawdown: {results[0].analyzers.drawdown.get_analysis()['max']['drawdown']:.2f}%"
    )
    return (
        round(results[0].analyzers.returns.get_analysis()["rnorm100"], 4),
        results[0].analyzers.drawdown.get_analysis()["max"]["drawdown"],
        results[0],
    )

index = 41

half_life = stationary_df[stationary_df['i'] == index]['half_life'].iloc[-1]
selected_row = stationary_df[stationary_df['i'] == index].iloc[-1]
hedge_ratio = convert_hedge_ratio(selected_row['hedge_ratio'])
print(selected_row, hedge_ratio, half_life)

nb_symbols = len(hedge_ratio)
pair = [selected_row[f'sid_{i + 1}'] for i in range(nb_symbols)]
print(pair)

fixed_weight = False
position_size = int(300 / max(hedge_ratio))

for symbol in pair:
    if symbol not in data:
        df_ = get_candles(base=symbol, quote='USDT', start_date=fromdate, interval=timeframe)
        data[symbol] = df_

ret, dd, res = run(
    data, 
    pair, 
    hedge_ratio, 
    half_life, 
    start_trading_at, 
    nb_symbols, 
    position_size=position_size, 
    fixed_weight=fixed_weight,
    debug=True,
    upper_limit=3,
    upper_limit2=5,
    middle=0.5,
    lower_limit=-3,
    lower_limit2=-5,
    enable_tp_sl=False
)

i                                       41
pairs                       ['MANA' 'UNI']
half_life                               61
hedge_ratio    [ 49.52048185 -34.10343683]
sid_1                                 MANA
sid_2                                  UNI
Name: 3, dtype: object [49.52048185, -34.10343683] 61
['MANA', 'UNI']
Starting Portfolio Value: 1000.00
2023-04-15T02:30:00, Start trade with ['MANA', 'UNI'], [49.52048185, -34.10343683], 61, 2
2023-03-18T23:15:00, Long spread zscore -3.1150613291820073 > -3, 0.6215000152587891 - 6.427999973297119
2023-03-18T23:15:00, Buy MANA, size = 478.07382752240113, cost = 297.1228911
2023-03-18T23:15:00, Sell UNI, size = -31.83270408058882, cost = 204.62062098
2023-03-18T23:30:00, BUY COMPLETE MANA, price: 0.6215, size: 478.07382752240113, cost 297.1229
2023-03-18T23:30:00, SELL COMPLETE UNI, price: 6.4270, size: -31.83270408058882cost -204.5888
2023-03-19T01:15:00, Take profit 0.15869629145304417 <> 0
2023-03-19T01:30:00, SELL COMPLETE MANA, 

In [12]:
import traceback

result = []
analyzers = {}


for i, row in stationary_df.head(10).iterrows():
    try:
        index = row.i

        half_life = stationary_df[stationary_df['i'] == index]['half_life'].iloc[-1]
        selected_row = stationary_df[stationary_df['i'] == index].iloc[-1]
        hedge_ratio = convert_hedge_ratio(selected_row['hedge_ratio'])
        print(selected_row, hedge_ratio, half_life)

        nb_symbols = len(hedge_ratio)
        pair = [selected_row[f'sid_{i + 1}'] for i in range(nb_symbols)]
        print(pair)
        
        fixed_weight = False
        position_size = int(200 / max(hedge_ratio))

        # fixed_weight = True
        # position_size = 500

        for symbol in pair:
            if symbol not in data:
                df_ = get_candles(base=symbol, quote='USDT', start_date=fromdate, interval=timeframe)
                data[symbol] = df_
            
        ret, dd, res = run(
            data, 
            pair, 
            hedge_ratio, 
            half_life, 
            start_trading_at, 
            nb_symbols, 
            position_size=position_size, 
            fixed_weight=fixed_weight,
            debug=False
        )
        
        result.append({
            'pair': pair,
            'hedge_ratio': hedge_ratio,
            'half_life': half_life,
            'ret': ret,
            'dd': dd,
        })
        analyzers[','.join(pair)] = res
        print('========================================================')
    except KeyboardInterrupt:
        break
    except:
        # traceback.print_exc()
        pass


i                                       12
pairs                        ['LTC' 'XMR']
half_life                               40
hedge_ratio    [ 26.26102528 -70.21746029]
sid_1                                  LTC
sid_2                                  XMR
Name: 0, dtype: object [26.26102528, -70.21746029] 40
['LTC', 'XMR']
Starting Portfolio Value: 1000.00DT 15m 2023-04-15 09:30:00                    
Trade Analysis Results:
               Total Open     Total Closed   Total Won      Total Lost     
               0              120            66             54             
               Strike Rate    Win Streak     Losing Streak  PnL Net        
               55.000000000000015              3              17.6           
Final Portfolio Value: 1017.60
Norm. Annual Return: 15.24%
Max Drawdown: 4.16%
i                                     34
pairs                    ['MATIC' 'XMR']
half_life                             49
hedge_ratio    [22.88928805 35.38803656]
sid_1               

In [13]:

result_df = pd.DataFrame(result)
result_df['ret'] = result_df['ret'].astype('float32')
result_df.sort_values(by=['ret', 'dd'], ascending=[False, True], inplace=True)
result_df

,pair,hedge_ratio,half_life,ret,dd
6,"[XRP, XMR]","[4.83818405, -48.98621273]",63,184.837494,16.252773
8,"[BTC, XMR]","[10.67405194, -49.21769878]",69,130.332703,3.736246
7,"[XMR, MANA]","[27.97782191, 18.06431601]",65,80.811203,2.197904
1,"[MATIC, XMR]","[22.88928805, 35.38803656]",49,51.170101,1.565054
2,"[XMR, UNI]","[37.72579497, 18.23129893]",53,36.949699,1.503346
3,"[MANA, UNI]","[49.52048185, -34.10343683]",61,27.673000,1.693996
9,"[LTC, MATIC]","[14.58328594, 26.82504414]",69,17.667900,3.475376
5,"[BNB, XMR]","[20.62332548, 36.70043684]",62,17.237200,2.176427
0,"[LTC, XMR]","[26.26102528, -70.21746029]",40,15.242000,4.155662
4,"[NEAR, MANA]","[28.55084246, -50.96965568]",61,0.569000,4.200326


# RESULT

In [14]:
# res = analyzers['BTC,UNI']

# # If no name has been specified, the name is the class name lowercased
# return_analyzer = res.analyzers.getbyname('return')
# # print(tret_analyzer.get_analysis())
# account_return = pd.Series(return_analyzer.get_analysis(), name='return')
# cum_r = (1 + account_return).cumprod()
# plt.axvline(x = start_trading_at, color = 'b', label = 'OOS')

# cum_r.plot()
# plt.show()
